In [2]:
# %load torch_LSTM_REG.py
#%% ============================import============================
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
import joblib
import numpy as np
import pandas as pd
import os

In [3]:


#%% ============================load data============================
# 設定 LSTM 往前看的筆數和預測筆數
LookBackNum = 12
ForecastNum = 48

# 載入訓練資料
DataName = os.getcwd() + '/ExampleTrainData(AVG)/AvgDATA_17.csv'
SourceData = pd.read_csv(DataName, encoding='utf-8')

# 迴歸分析用資料
Regression_X_train = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values
Regression_y_train = SourceData[['Power(mW)']].values

# LSTM 用資料
AllOutPut = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values

# 正規化
LSTM_MinMaxModel = MinMaxScaler().fit(AllOutPut)
AllOutPut_MinMax = LSTM_MinMaxModel.transform(AllOutPut)

X_train = []
y_train = []

for i in range(LookBackNum, len(AllOutPut_MinMax)):
    X_train.append(AllOutPut_MinMax[i - LookBackNum:i, :])
    y_train.append(AllOutPut_MinMax[i, :])

X_train = np.array(X_train)
y_train = np.array(y_train)


In [4]:
#%% ============================定義 PyTorch Dataset============================
class WeatherDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


train_dataset = WeatherDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [5]:
#%% ============================建置 LSTM 模型============================
class WeatherLSTM(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size):
        super(WeatherLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_sizes[0], batch_first=True)
        self.lstm2 = nn.LSTM(hidden_sizes[0], hidden_sizes[1], batch_first=True)
        self.lstm3 = nn.LSTM(hidden_sizes[1], hidden_sizes[2], batch_first=True)
        self.lstm4 = nn.LSTM(hidden_sizes[2], hidden_sizes[3], batch_first=True)
        self.fc = nn.Linear(hidden_sizes[3], output_size)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x, _ = self.lstm4(x)
        x = self.dropout(x[:, -1, :])  # 取最後一個時間步的輸出
        x = self.fc(x)
        return x

In [6]:
#%%============================device============================
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [7]:
#%%============================initial model============================
# 模型初始化
input_size = 5
hidden_sizes = [512, 256, 128, 32]
output_size = 5
model = WeatherLSTM(input_size, hidden_sizes, output_size).to(device)

lambda_l1 = 1e-5  # L1 正则化系数
lambda_l2 = 1e-4  # L2 正则化系数
#============================訓練模型============================
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=lambda_l2)  # L2 正则化

# 訓練
epochs = 100
for epoch in range(epochs):
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
          # 移動資料到 GPU
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)

        l1_regularization = sum(torch.sum(torch.abs(param)) for param in model.parameters())
        loss += lambda_l1 * l1_regularization
        
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(train_loader)}")

# 保存模型
torch.save(model.state_dict(), f'WeatherLSTM_{pd.Timestamp.now().strftime("%Y-%m-%dT%H_%M_%SZ")}.pth')
print("LSTM Model Saved")

Epoch 1/100, Loss: 0.4286292816807584
Epoch 2/100, Loss: 0.162674078127233
Epoch 3/100, Loss: 0.10684981233463055
Epoch 4/100, Loss: 0.082090329833147
Epoch 5/100, Loss: 0.06430500927494794
Epoch 6/100, Loss: 0.05582416857161173
Epoch 7/100, Loss: 0.051033507305674436
Epoch 8/100, Loss: 0.046360711771540525
Epoch 9/100, Loss: 0.04283840945217668
Epoch 10/100, Loss: 0.04081233499980554
Epoch 11/100, Loss: 0.0381209036199058
Epoch 12/100, Loss: 0.035912410424249926
Epoch 13/100, Loss: 0.03503639741641719
Epoch 14/100, Loss: 0.03352504409849644
Epoch 15/100, Loss: 0.03246055448018923
Epoch 16/100, Loss: 0.031601531203927065
Epoch 17/100, Loss: 0.030494513580711875
Epoch 18/100, Loss: 0.02959448053705983
Epoch 19/100, Loss: 0.029084371630011534
Epoch 20/100, Loss: 0.02832952664211029
Epoch 21/100, Loss: 0.02784516639644053
Epoch 22/100, Loss: 0.027377169306685285
Epoch 23/100, Loss: 0.02668928305005155
Epoch 24/100, Loss: 0.026189915381553696
Epoch 25/100, Loss: 0.025809808338924153
Epoch 

In [8]:

#%% ============================回歸模型============================
RegressionModel = LinearRegression()
RegressionModel.fit(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train)

# 保存回歸模型
joblib.dump(RegressionModel, f'WeatherRegression_{pd.Timestamp.now().strftime("%Y-%m-%dT%H_%M_%SZ")}')
print("Regression Model Saved")
print("Intercept:", RegressionModel.intercept_)
print("Coefficients:", RegressionModel.coef_)
print("R-squared:", RegressionModel.score(LSTM_MinMaxModel.transform(Regression_X_train), Regression_y_train))


Regression Model Saved
Intercept: [-221.39953665]
Coefficients: [[   0.          135.11381615  117.68809059   23.89564382 1761.56873922]]
R-squared: 0.899519098844525


In [ ]:

#%% ============================預測============================
# 載入模型
import torch

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
##記得要改模型名稱
# 載入 LSTM 模型
model = WeatherLSTM(input_size=5, hidden_sizes=[512, 256, 128, 32], output_size=5)
model.load_state_dict(torch.load('WeatherLSTM_2024-11-27T16_00_12Z.pth',weights_only=True))
model.to(device)
model.eval()

# 載入迴歸模型
RegressionModel = joblib.load('WeatherRegression_2024-11-27T16_09_45Z')

# 載入測試資料
DataName = os.getcwd() + r"/ExampleTestData/upload.csv"
SourceData = pd.read_csv(DataName, encoding='utf-8')
target = ['序號']
EXquestion = SourceData[target].values

PredictOutput = []  # 存放預測值 (天氣參數)
PredictPower = []   # 存放預測值 (發電量)

count = 0
while count < len(EXquestion):
    print("count :", count)
    LocationCode = int(EXquestion[count].item())
    strLocationCode = str(LocationCode)[-2:]
    if LocationCode < 10:
        strLocationCode = "0" + str(LocationCode)

    DataName = os.getcwd() + f"/ExampleTrainData(IncompleteAVG)/IncompleteAvgDATA_{strLocationCode}.csv"
    SourceData = pd.read_csv(DataName, encoding="utf-8")
    ReferTitle = SourceData[['Serial']].values
    ReferData = SourceData[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']].values

    inputs = []  # 存放參考資料
    # 找到相同的一天，把 12 個資料都加進 inputs
    for DaysCount in range(len(ReferTitle)):
        if str(int(ReferTitle[DaysCount].item()))[:8] == str(int(EXquestion[count].item()))[:8]:
            TempData = ReferData[DaysCount].reshape(1, -1)  # Shape: (1, 5)
            TempData = LSTM_MinMaxModel.transform(TempData)
            TempData = TempData.squeeze(0)  # 去掉第一维，形状变为 (5,)
            inputs.append(TempData)

            # print("TempData shape:", TempData.shape)  # 應為 (1, 5)

    inputs = torch.tensor(inputs, dtype=torch.float32).to(device)  # 將 inputs 轉成 PyTorch Tensor
    # 預測
    with torch.no_grad():  # 禁止梯度计算
        for i in range(ForecastNum):
            # 将新的预测值加入参考数据 (用自己的预测值往前看)
            if i > 0:
                new_prediction = torch.tensor(PredictOutput[i - 1], dtype=torch.float32).unsqueeze(0).to(device)  # Shape: (1, 5)
                inputs = torch.cat((inputs, new_prediction), dim=0)  # 拼接 -> Shape: (N+1, 5)

            # 切出新的参考数据 12 笔 (往前看 12 笔)
            X_test = inputs[-LookBackNum:]  # Shape: (LookBackNum, 5)
            X_test = X_test.unsqueeze(0)  # Add batch dimension -> Shape: (1, LookBackNum, 5)
            # print("X_test shape:", X_test.shape)  # 应为 (1, LookBackNum, 5)

            # 前向传播
            predicted = model(X_test)  # Shape: (1, 5)
            PredictOutput.append(predicted.cpu().numpy().flatten())  # 添加预测值

            # 使用回归模型预测发电量
            predicted_power = RegressionModel.predict(predicted.cpu().numpy())
            PredictPower.append(np.round(predicted_power, 2).flatten())


    count += 48  # 每次預測都要預測 48 個，因此加 48 個會切到下一天

print("Prediction Completed")

# %%
#寫預測結果寫成新的CSV檔案
# 將陣列轉換為 DataFrame
df = pd.DataFrame(PredictPower, columns=['答案'])

# 將 DataFrame 寫入 CSV 檔案
df_q = pd.read_csv('upload(no answer).csv')
output_df = pd.concat([df_q['序號'], df], axis=1)
output_df.to_csv('output1.csv', index=False)
print('Output CSV File Saved')




FileNotFoundError: [Errno 2] No such file or directory: 'WeatherLSTM_2024-11-27T16_00_12Z.pth'

In [ ]:
# %%
#寫預測結果寫成新的CSV檔案
# 將陣列轉換為 DataFrame
df = pd.DataFrame(PredictPower, columns=['答案'])

# 將 DataFrame 寫入 CSV 檔案
df_q = pd.read_csv('upload(no answer).csv')
output_df = pd.concat([df_q['序號'], df], axis=1)
output_df.to_csv('output1.csv', index=False)
print('Output CSV File Saved')